<a href="https://colab.research.google.com/github/Kush-agra-wal/DocBot_Yamaha/blob/main/hackathon_document_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
from google.colab import userdata
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
from huggingface_hub import login
token=userdata.get('token')
login(token=token)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
! pip install langchain unstructured[all-docs] pydantic lxml
! pip install tensorflow-probability imageio pillow
! pip install tesseract
! pip install ctransformers
! pip install GPT4AllEmbeddings
! pip install gpt4all
! pip install chromadb

  Using cached tesseract-0.1.3.tar.gz (45.6 MB)
  Preparing metadata (setup.py) ... done
  Created wheel for tesseract: filename=tesseract-0.1.3-py3-none-any.whl size=45562552 sha256=29fa358a413569be12c21054471a6ec21e88cbdbcc9802718f179a98e52bcd76
  Stored in directory: /root/.cache/pip/wheels/71/c9/aa/698c579693e83fdda9ad6d6f0d8f61ed986e27925ef576f109
Successfully built tesseract
ERROR: Could not find a version that satisfies the requirement GPT4AllEmbeddings (from versions: none)
ERROR: No matching distribution found for GPT4AllEmbeddings
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/4

In [4]:
path="/content/drive/MyDrive/Datasets/DocBot/english/"


In [5]:
!sudo apt update

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [52.9 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,755 kB]
Get:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-

In [6]:
!sudo apt-get install libleptonica-dev tesseract-ocr libtesseract-dev python3-pil tesseract-ocr-eng tesseract-ocr-script-latn poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libarchive-dev libimagequant0 libraqm0 mailcap mime-support python3-olefile
  tesseract-ocr-osd
Suggested packages:
  python-pil-doc
The following NEW packages will be installed:
  libarchive-dev libimagequant0 libleptonica-dev libraqm0 libtesseract-dev
  mailcap mime-support poppler-utils python3-olefile python3-pil tesseract-ocr
  tesseract-ocr-eng tesseract-ocr-osd tesseract-ocr-script-latn
0 upgraded, 14 newly installed, 0 to remove and 50 not upgraded.
Need to get 40.2 MB of archives.
After this operation, 124 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libarchive-dev amd64 3.6.0-1ubuntu1 [581 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libimagequant0 amd64 2.17.0-1 [34.6 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libleptonica-dev amd64 1.82.

In [7]:
from typing import Any

from pydantic import BaseModel
from unstructured.partition.pdf import partition_pdf

# Get elements
raw_pdf_elements = partition_pdf(
    filename=path + "sample_pdf.pdf",
    # Using pdf format to find embedded image blocks
    extract_images_in_pdf=True,
    # Use layout model (YOLOX) to get bounding boxes (for tables) and find titles
    # Titles are any sub-section of the document
    infer_table_structure=True,
    # Post processing to aggregate text once we have the title
    chunking_strategy="by_title",
    # Chunking params to aggregate text blocks
    # Attempt to create a new chunk 3800 chars
    # Attempt to keep chunks > 2000 chars
    # Hard max on chunks
    max_characters=1850,
    new_after_n_chars=1750,
    combine_text_under_n_chars=1000,
    image_output_dir_path=path,
)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


yolox_l0.05.onnx:   0%|          | 0.00/217M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/115M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
# Create a dictionary to store counts of each type
category_counts = {}

for element in raw_pdf_elements:
    category = str(type(element))
    if category in category_counts:
        category_counts[category] += 1
    else:
        category_counts[category] = 1

# Unique_categories will have unique elements
unique_categories = set(category_counts.keys())
category_counts

{"<class 'unstructured.documents.elements.CompositeElement'>": 13,
 "<class 'unstructured.documents.elements.Table'>": 1}

In [9]:
# i=0
# for element in raw_pdf_elements:
#     if "unstructured.documents.elements.Image" in str(type(element)):
#         print(i)
#         break
#     i+=1

In [10]:
raw_pdf_elements[0].metadata.orig_elements[1].to_dict()

{'type': 'NarrativeText',
 'element_id': '1aea996e-a7f3-4a39-9bcd-8569cec40e9f',
 'text': 'YRC (Yamaha Ride Control) Yamaha Ride Control is a system that incorporates numerous sensors and controls to support an improved riding experience. The vehicle senses and can react to forces along the longitudi- nal (front-to-back), lateral (left-to-right), and vertical (up-and-down) axes. Lean angle and G-force accelerations are also detected. This information is proc- essed multiple times a second and the related physical systems are automati- cally adjusted as necessary. The fol- lowing functions represent individual YRC items which can be turned on/off or adjusted to suit various riders and riding conditions. For setting details, see pages 4-15 and 4-19.',
 'metadata': {'detection_class_prob': 0.9282627701759338,
  'coordinates': {'points': ((173.21183333333323, 229.26394444444438),
    (173.21183333333323, 829.2650555555554),
    (647.74169921875, 829.2650555555554),
    (647.74169921875, 22

In [11]:
for ele in raw_pdf_elements:
  print(ele.to_dict())

{'type': 'CompositeElement', 'element_id': '4144985935e2790df2ecb4e1df843240', 'text': 'EAU93653\n\nYRC (Yamaha Ride Control) Yamaha Ride Control is a system that incorporates numerous sensors and controls to support an improved riding experience. The vehicle senses and can react to forces along the longitudi- nal (front-to-back), lateral (left-to-right), and vertical (up-and-down) axes. Lean angle and G-force accelerations are also detected. This information is proc- essed multiple times a second and the related physical systems are automati- cally adjusted as necessary. The fol- lowing functions represent individual YRC items which can be turned on/off or adjusted to suit various riders and riding conditions. For setting details, see pages 4-15 and 4-19.\n\nEWA18221\n\nA WARNING\n\nThe Yamaha Ride Control (YRC) sys- tem is not a substitute for the use of proper riding techniques or the ex- pertise of the operator. This system cannot prevent loss of control caused by rider errors such

In [12]:
unchunked_list = []

for ele in raw_pdf_elements:
    sublist = []
    for orig_ele in ele.metadata.orig_elements:
        metadata = orig_ele.metadata.to_dict()
        filtered_dict = {}
        filtered_dict['text'] = orig_ele.to_dict()['text']
        filtered_dict['type'] = orig_ele.to_dict()['type']
        filtered_dict['page_number'] = metadata['page_number']
        if 'image_path' in metadata:
            filtered_dict['image_path']=metadata['image_path']
        sublist.append(filtered_dict)
    unchunked_list.append(sublist)
print(len(unchunked_list))

chunk_text_list = [i.text for i in raw_pdf_elements]
chunk_text_list[3]

14


'EWaz08e0\n\nEWA20880\n\nMake sure the engine has sufficient- ly slowed before shifting to a lower gear. Engaging a lower gear when the engine speed is too high could make the rear wheel lose traction. This could cause loss of control, an accident and injury. It could also cause engine or drivetrain damage.\n\nQuick shifter The quick shifter allows for clutch lev- er-less, electronically-assisted shifting.\n\n—p—\n\nWhen the sensor on the shift rod de- tects the appropriate motion in the shift pedal, engine power output is momen- tarily adjusted to allow for the gear change to occur. The quick shifter does not operate when the clutch lever is pulled, there- fore normal shifting can be done even when the quick shifter is set to on. Check the quick shifter indicator for current status and usability informa- tion.'

In [13]:
len(unchunked_list[0])
# raw_pdf_elements[0].metadata.orig_elements[10].to_dict()

9

In [14]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
# from langchain_openai import ChatOpenAI

import pathlib
import textwrap
import google.generativeai as genai
from IPython.display import display
from IPython.display import Markdown
# Used to securely store your API key

from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from ctransformers import AutoModelForCausalLM as cAutoModelForCausalLM
from ctransformers import AutoConfig, Config

In [15]:
conf = AutoConfig(Config(temperature=0.8, repetition_penalty=1.1,
                         batch_size=52, max_new_tokens=2048,
                         context_length=2048,gpu_layers=300))
llm = cAutoModelForCausalLM.from_pretrained("TheBloke/Mistral-7B-Instruct-v0.2-GGUF", model_type="mistral", config=conf)

def generate_output(question):
    formatted_input = f"<s> <INST> {question} </INST>"
    output = llm(formatted_input,repetition_penalty=1.15, max_new_tokens=2048)

    return output

def prompt_input(input_text):
    prompt_text = """You are an assistant tasked with summarizing tables and text. \
Give a concise summary of the table or text. Table or text chunk: {element} """
    return prompt_text.format(element=input_text)

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

mistral-7b-instruct-v0.2.Q2_K.gguf:   0%|          | 0.00/3.08G [00:00<?, ?B/s]

In [16]:
lambda_chain = lambda input_list: list(map(lambda text: generate_output(prompt_input(text)), input_list))
# Apply to text
text_summaries = lambda_chain(chunk_text_list)

In [17]:
text_summaries

[" The Yamaha Ride Control (YRC) is a motorcycle system that uses sensors to improve riding experience by reacting to forces along the longitudinal, lateral, and vertical axes, as well as lean angle and G-force accelerations. Functions of YRC can be adjusted or turned on/off for different riders and conditions. The YRC is not a substitute for proper riding techniques or expertise. There's also a warning against relying solely on the system to prevent loss of control due to rider errors, such as excessive speed or sharp turns. Additionally, there's a Power Delivery mode system with four different control maps that regulate throttle valve opening based on degree of grip operation for various preferences and environments.",
 " This text describes a special feature of a vehicle's traction control system. The system helps maintain traction during acceleration by regulating engine power when sensors detect rear wheel slippage. It can adjust according to the lean angle, applying less control 

In [18]:
import uuid

from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryStore
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document

# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="summaries", embedding_function=GPT4AllEmbeddings()
)

# The storage layer for the parent documents
store = InMemoryStore()  # <- Can we extend this to images
id_key = "doc_id"

# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=store,
    id_key=id_key,
)

Downloading: 100%|██████████| 45.9M/45.9M [00:00<00:00, 122MiB/s]
Verifying: 100%|██████████| 45.9M/45.9M [00:00<00:00, 551MiB/s]


In [19]:
# Add texts
doc_ids = [str(uuid.uuid4()) for _ in text_summaries]
summary_texts = [
    Document(page_content=s, metadata={id_key: doc_ids[i]})
    for i, s in enumerate(text_summaries)
]
retriever.vectorstore.add_documents(summary_texts)
retriever.docstore.mset(list(zip(doc_ids, unchunked_list)))


In [ ]:
user_q="what is engine brake management"
retrieved_query_vectorstore=retriever.vectorstore.similarity_search(user_q)
retrieved_query_vectorstore

In [ ]:
doc_id1=retrieved_query_vectorstore[0].__dict__['metadata']['doc_id']
doc_id2=retrieved_query_vectorstore[1].__dict__['metadata']['doc_id']

In [ ]:

def generate_output(question):
    formatted_input = f"<s> <INST> {question} </INST>"
    output = llm(formatted_input,repetition_penalty=1.15, max_new_tokens=2048)
    return output

def prompt_input(user_q, retrieved_info1,retrieved_info2):
    prompt_text = """
        The user is asking about the given question.
        User Question:{user_q}
        Information1:{retrieved_info1}
        Information2:{retrieved_info2}
        The information provided to you is in the form of a list where each element in the list is a dictionary containing the information having text, page_number, and type.
        Prompt Explanation for LLMs:
        Tasks:
        Task 1: Based on the provided information, generate a concise and accurate response to the user's question.
        Task 2: If applicable, locate and provide the image_path of the closest image element from the text which answers the user's question.
        Task 3: Provide page number from which the text is taken, you can provide multiple page numbers if applicable
        Instructions:
        Use the informations provided to craft a response that directly addresses the user's question.
        If there are multiple relevant sections, prioritize the most pertinent ones for the response.
        Ensure that the response is clear, coherent, and free from ambiguity.
        If an image is available, provide its path only if it directly relates to the user's question or enhances understanding of the topic. You have to return the image_path of the image that comes right after the used text in the list as usually it is the image related to that text.
        give the output in the following json format: output: your-output, image_path:image path for closest image or just none if it is not right after the text you have used, page_number: <the page number fetched>
        Sample Example Output(in json):

          "output": "Cruise control system model is equipped with a cruise control system designed to maintain a set cruising speed without the need to hold open the throttle by hand.",
          "image_path":"/mnt/d/ME/Classes/Sem_8/DL/Hackathon/figures/figure-6-7.jpg",
          "page_number":"6"

        Use the image_path and page_number, given in the information1 & information2 as it is.
        """
    return prompt_text.format(user_q=user_q, retrieved_info1=retrieved_info1,retrieved_info2=retrieved_info2)


In [ ]:
prompt=prompt_input(user_q=user_q, retrieved_info1=retriever.docstore.store[doc_id1],retrieved_info2=retriever.docstore.store[doc_id2])
output=generate_output(prompt)

In [ ]:
output

In [ ]:
import json
sus=json.loads(output)

In [ ]:
sus

In [ ]:
sus['image_path']